In [1]:
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

import numpy as np

from sklearn.linear_model import LogisticRegression

import logging
import sys
import random

In [30]:
#import os
#files = os.listdir('unsup')
#unsup_texts = open('train-unsup.txt', 'w+')
#for file in files:
    #with open('unsup/'+file) as fi:
        #text = fi.read()
        #unsup_texts.write(text+'\n')
        
#unsup_texts.close()


In [31]:
#sentences = []
#with utils.smart_open('train-unsup.txt') as fin:
    #for item_no, line in enumerate(fin):
        # TaggedDocument([word1, word2 ...], [tagx])
        #sentences.append(utils.to_unicode(line).split())

In [20]:
# code from the tutorial of the python model logging.
# create a logger, the same name corresponding to the same logger.
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to info
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)

# create formatter and add formatter to ch
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)

logger.addHandler(ch)   # add ch to the logger

In [34]:
## the code for the doc2vec
class TaggedLineSentence(object):
    """
    sources: [file1 name: tag1 name, file2 name: tag2 name ...]
    privade two functions:
        to_array: transfer each line to a object of TaggedDocument and then add to a list
        perm: permutations
    """
    def __init__(self, sources):
        self.sources = sources

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            source = 'data/' + source
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    # TaggedDocument([word1, word2 ...], [tagx])
                    self.sentences.append(TaggedDocument(utils.to_unicode(line).split(),
                                       [prefix + '_%s' % item_no]))
        return self.sentences

    def perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)    # Note that this line does not return anything.
        return shuffled

In [35]:
sources = {'test-neg.txt': 'TEST_NEG', 'test-pos.txt': 'TEST_POS',
           'train-neg.txt': 'TRAIN_NEG','train-pos.txt': 'TRAIN_POS',
           'train-unsup.txt': 'TRAIN_UNS'}
sentences = TaggedLineSentence(sources)

In [43]:
# set the parameter and get a model.
# by default dm=1, PV-DM is used. Otherwise, PV-DBOW is employed.
model = Doc2Vec(min_count=1, window=10, vector_size=100,
                sample=1e-4, negative=5, dm=1, workers=7)
model.build_vocab(sentences.to_array())

# train the model
for epoch in range(5):
    logger.info('epoch %d' % epoch)
    model.train(sentences.perm(),
                total_examples=model.corpus_count,
                epochs=model.iter
                )

logger.info('model saved')
model.save('./imdb.d2v')

2018-06-22 20:19:21,685 - gensim.models.doc2vec - INFO - collecting all words and their counts
2018-06-22 20:19:21,685 - gensim.models.doc2vec - INFO - collecting all words and their counts
2018-06-22 20:19:21,687 - gensim.models.doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-06-22 20:19:21,687 - gensim.models.doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-06-22 20:19:22,180 - gensim.models.doc2vec - INFO - PROGRESS: at example #10000, processed 2291061 words (4664125/s), 47469 word types, 10000 tags
2018-06-22 20:19:22,180 - gensim.models.doc2vec - INFO - PROGRESS: at example #10000, processed 2291061 words (4664125/s), 47469 word types, 10000 tags
2018-06-22 20:19:22,702 - gensim.models.doc2vec - INFO - PROGRESS: at example #20000, processed 4647849 words (4530936/s), 66058 word types, 20000 tags
2018-06-22 20:19:22,702 - gensim.models.doc2vec - INFO - PROGRESS: at example #20000, processed 46

/Users/richardsun/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


2018-06-22 20:19:42,341 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 3.47% examples, 433893 words/s, in_qsize 13, out_qsize 0
2018-06-22 20:19:42,341 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 3.47% examples, 433893 words/s, in_qsize 13, out_qsize 0
2018-06-22 20:19:43,361 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 7.37% examples, 464447 words/s, in_qsize 13, out_qsize 0
2018-06-22 20:19:43,361 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 7.37% examples, 464447 words/s, in_qsize 13, out_qsize 0
2018-06-22 20:19:44,371 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 11.10% examples, 465504 words/s, in_qsize 13, out_qsize 0
2018-06-22 20:19:44,371 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 11.10% examples, 465504 words/s, in_qsize 13, out_qsize 0
2018-06-22 20:19:45,373 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 14.64% examples, 461922 words/s, in_qsize 13, out_qsize 0
201

In [44]:
# load and test the model
logger.info('model loaded')
model = Doc2Vec.load('./imdb.d2v')

logger.info('Sentiment Analysis...')

logger.info('transfer the train document to the vector')
train_arrays = np.zeros((25000, 100))
train_labels = np.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    # note that the attribute is model.docvecs
    train_arrays[i], train_arrays[12500+i] = \
        model.docvecs[prefix_train_pos], model.docvecs[prefix_train_neg]
    train_labels[i], train_labels[12500+i] = 1, 0


logger.info('transfer the test document to the vector')
test_arrays = np.zeros((25000, 100))
test_labels = np.zeros(25000)

2018-06-22 20:36:29,091 - root - INFO - model loaded
2018-06-22 20:36:29,091 - root - INFO - model loaded
2018-06-22 20:36:29,095 - gensim.utils - INFO - loading Doc2Vec object from ./imdb.d2v
2018-06-22 20:36:29,095 - gensim.utils - INFO - loading Doc2Vec object from ./imdb.d2v
2018-06-22 20:36:38,670 - gensim.utils - INFO - loading vocabulary recursively from ./imdb.d2v.vocabulary.* with mmap=None
2018-06-22 20:36:38,670 - gensim.utils - INFO - loading vocabulary recursively from ./imdb.d2v.vocabulary.* with mmap=None
2018-06-22 20:36:38,673 - gensim.utils - INFO - loading trainables recursively from ./imdb.d2v.trainables.* with mmap=None
2018-06-22 20:36:38,673 - gensim.utils - INFO - loading trainables recursively from ./imdb.d2v.trainables.* with mmap=None
2018-06-22 20:36:38,674 - gensim.utils - INFO - loading syn1neg from ./imdb.d2v.trainables.syn1neg.npy with mmap=None
2018-06-22 20:36:38,674 - gensim.utils - INFO - loading syn1neg from ./imdb.d2v.trainables.syn1neg.npy with mm

In [45]:
for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i], test_arrays[12500 + i] = \
        model.docvecs[prefix_test_pos], model.docvecs[prefix_test_neg]
    test_labels[i], test_labels[12500 + i] = 1, 0

logger.info('Fitting')
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

print("accuracy: "+ str(classifier.score(test_arrays, test_labels)))

2018-06-22 20:36:42,422 - root - INFO - Fitting
2018-06-22 20:36:42,422 - root - INFO - Fitting
accuracy: 0.84784
